Docking Inputs (Files should be in root dir relative to this notebook)

In [4]:
from pymol import cmd
import py3Dmol
import pandas as pd
import random
from openbabel import pybel
import networkx as nx

from matplotlib import cm, colors
from matplotlib import pyplot as plt
from IPython.display import IFrame
import math
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

import sys, os, random
sys.path.insert(1, 'utilities/')

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import py3Dmol
from rdkit.Geometry import Point3D
from MDAnalysis import Merge

#import prolif_old as plf
#from prolif_old.plotting.network import LigNetwork

from utils import getbox, generate_ledock_file, dok_to_sdf,fix_protein

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False


#The receptor file
receptor_file='4TVJ_clean.pdbqt' 

#the csv file containing the ligand smiles and ID info
ligand_file='MainLigands_PARP.csv'

#The PDB ID 
pdbID='4TVJ-Ex32a' 

#Define Gridbox
##Note the use of dicts here is for easy cross-compatablility with SMINA; vina will use only a list of the dict values

# Native GridBox (GB0)
#PARP2_center={'center_x': 18.833499908447266, 'center_y': -2.2054998874664307, 'center_z': 17.501500129699707}
#PARP2_size={'size_x': 17.152999877929688, 'size_y': 20.658999919891357, 'size_z': 19.641000747680664}

# GridBox 1 (GB1)
PARP2_center={'center_x': 22.54, 'center_y': 2.66, 'center_z': 14.99}
PARP2_size={'size_x': 24.57, 'size_y': 30.59, 'size_z': 21.41}


Preprocessing for virtual screen

In [5]:
from vina import Vina
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy
from rdkit import Chem
from rdkit.Chem import rdDistGeom
import pandas as pd

v = Vina(sf_name='vina')

#Extract ligand information from input csv file
ligand_df=pd.read_csv(ligand_file)

#Make lists from the ligand df
ligand_smiles=ligand_df['Smiles'].values.tolist()
ligand_IDs=ligand_df['ID'].values.tolist()

#zip the lists into a dict
ligand_dict=dict(zip(ligand_smiles,ligand_IDs))

#Set the receptor in vina
v.set_receptor(receptor_file)

#Make the gridmaps
v.compute_vina_maps(center=list(PARP2_center.values()), box_size=list(PARP2_size.values()))


Computing Vina grid ... done.


In [6]:
#Guarentees the conformer generation for initializing a ligands 3D coordinates is suscessful
def get_real_conformer(mol):
    conf_failure=0
    comf_attempts=0
    while conf_failure==0:
        rdDistGeom.EmbedMolecule(mol)
        conf_failure=mol.GetNumConformers()
        comf_attempts+=1
        print(str(comf_attempts) + " failed conformer generations. Reattempting...")
        if conf_failure==1:
            break
    return(mol)


for smiles,id in ligand_dict.items():
    #create rdkit mol object from smiles
    mol = Chem.MolFromSmiles(smiles)

    #add polar Hydrogens to mol
    mol_H = Chem.AddHs(mol)

    #Embed the mol for use in meeko
    lig_mol=get_real_conformer(mol_H)
    print('Conformer generation successful.')

    #prepare the mol in meeko
    prep = MoleculePreparation()
    mol_prep = prep.prepare(lig_mol)

    #write the final pdbqt string for use in vina
    pdbqt_string = prep.write_pdbqt_string()
    #pdbqt_string = PDBQTWriterLegacy.write_string(mol_prep)

    #Set the ligand in vina
    v.set_ligand_from_string(pdbqt_string)

    #dock the ligand
    print('Docking ligand'+id+'...')
    v.dock(exhaustiveness=32, n_poses=20)

    #Save the docked poses to file
    v.write_poses(pdbID+'_'+id+'_vina_out.pdbqt', n_poses=10, overwrite=True)
   

1 failed conformer generations. Reattempting...
Conformer generation successful.
Docking ligandA45_B295-AdS12...
Performing docking (random seed: 489411304) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -10.38          0          0
   2       -10.25     0.6954      2.469
   3       -10.18      1.122      6.192
   4       -10.18      0.678      3.274
   5       -10.17     0.6924      3.074
   6       -10.13     0.7062      5.947
   7       -10.13     0.8535      3.122
   8       -10.12       1.77      6.977
   9       -10.12     0.9621      3.126
  10       -10.08      1.078      6.084
  11       -10.08     0.6676      3.683
  12       -10.05      1.036      6.091
  13       -10.05      1.018      2.445
  14       -10.04      1.438      6.